In [21]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

In [22]:
import os
for dirname, _, filenames in os.walk("glove-twitter"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

glove-twitter\glove.twitter.27B.100d.txt
glove-twitter\glove.twitter.27B.200d.txt
glove-twitter\glove.twitter.27B.25d.txt
glove-twitter\glove.twitter.27B.50d.txt


In [23]:
%%time
#twitter = pd.read_csv("preprocessed_data.csv")    #normal one used before
#twitter = pd.read_csv("sorted_preprocessed_data.csv")  #with lowest emotion scores
twitter = pd.read_csv("final_dataset.csv")
#twitter = pd.read_csv("greater_than_some_score.csv")

Wall time: 5.41 s


In [3]:
pd.set_option('display.max_rows', 1000)
twitter

,_score,tweet_id,text,identification,emotion,text_stemmed,Category
0,391,0x376b20,people who post add me on snapchat must be deh...,train,anticipation,peopl who post add me on snapchat must be dehy...,4
1,433,0x2d5350,brianklaas as we see trump is dangerous to fre...,train,sadness,brianklaa as we see trump is danger to freepre...,8
2,376,0x1cd5b0,now issa is stalking tasha 😂😂😂,train,fear,now issa is stalk tasha 😂😂😂,1
3,120,0x1d755c,riskshow thekevinallison thx for the best time...,train,joy,riskshow thekevinallison thx for the best time...,7
4,1021,0x2c91a8,still waiting on those supplies liscus,train,anticipation,still wait on those suppli liscu,4
...,...,...,...,...,...,...,...
1451386,94,0x321566,im so happy nowonder the name of this show hap...,train,joy,im so happi nowond the name of thi show happi ...,7
1451387,627,0x38959e,in every circumtance id like to be thankful to...,train,joy,in everi circumt id like to be thank to the al...,7
1451388,274,0x2cbca6,theres currently two girls walking around the ...,train,joy,there current two girl walk around the librari...,7
1451389,840,0x24faed,ah corporate life where you can date using ju...,train,joy,ah corpor life where you can date use just the...,7


In [5]:
docs = twitter["text_stemmed"]

In [49]:
labels = twitter["Category"]

In [6]:
docs

0          peopl who post add me on snapchat must be dehy...
1          brianklaa as we see trump is danger to freepre...
2                                now issa is stalk tasha 😂😂😂
3          riskshow thekevinallison thx for the best time...
4                           still wait on those suppli liscu
                                 ...                        
1451386    im so happi nowond the name of thi show happi ...
1451387    in everi circumt id like to be thank to the al...
1451388    there current two girl walk around the librari...
1451389    ah corpor life where you can date use just the...
1451390                           bless to be live sundayvib
Name: text_stemmed, Length: 1451391, dtype: object

In [24]:
#TOKENIZER
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
print(encoded_docs)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [25]:
#Creating Vectors Using Padding
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[13426  1217   144    15]
 [   32     4 72140   584]
 [    8  3622  2152   924]
 ...
 [   54   799  3334 29349]
 [ 1769    15   675    12]
 [    2    16   125 10698]]


In [26]:
len(padded_docs)

1451391

## Glove Word Embedding

In [42]:
# load the whole embedding into memory
embeddings_index = dict()
f = open("glove-twitter/glove.twitter.27B.200d.txt",encoding="utf8")
for line in f:
        values = line.split()
        word = values[0]
        coefs = asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 1193515 word vectors.


In [41]:
#continue here: https://www.kaggle.com/hassanamin/glove-based-text-classification

In [44]:
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 200))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [46]:
# define model
model = Sequential()
e = Embedding(vocab_size, 200, weights=[embedding_matrix], input_length=4, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 4, 200)            171581000 
                                                                 
 flatten (Flatten)           (None, 800)               0         
                                                                 
 dense (Dense)               (None, 1)                 801       
                                                                 
Total params: 171,581,801
Trainable params: 801
Non-trainable params: 171,581,000
_________________________________________________________________
None


In [55]:
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 4.385655


In [56]:
# import tensorflow as tf

# epochs = 50
# batch_size = 512 #512

# history = model.fit(padded_docs, labels, epochs=epochs, batch_size=batch_size)#, 
#                     #validation_split=0.1, callbacks=[tf.keras.callbacks.EarlyStopping
#                     #(monitor='val_loss', patience=1, min_delta=0.0001)])